In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors, KDTree
from scores import scores
import time
import warnings
warnings.filterwarnings("ignore")

### 4.) Import Data

In [6]:
%run ../notebooks/utils.ipynb

In [3]:
trainandknn_Xy_original_df = pd.read_csv("../data/train_new.csv", sep="|")
train_Xy_original_df, knn_Xy_original_df = train_test_split(trainandknn_Xy_original_df,train_size=0.75, random_state=42) # if FINAL_SUBMISSION else 0.8) #small
test_X_original_df  = pd.read_csv("../data/test.csv", sep="|").iloc[0:500] #TODO: For faster testing we use less data from the test set
test_final_X_df = pd.read_csv("../data/test.csv", sep="|")

train_Xy_wo_knn_df = pd.read_csv("../data/train_new.csv", sep="|")

#Only for test routines
val_Xy_original_df = pd.read_csv("../data/val_new.csv", sep="|")
train_complete_Xy_original_df = pd.read_csv("../data/train.csv", sep="|")

In [4]:
#convention for variables names: datasetname_columntype_transformstatus_dataframeornot
train_y_original_df = train_Xy_original_df[["fraud"]].copy()
train_X_original_df = train_Xy_original_df.copy().drop("fraud", axis=1)

knn_y_original_df = knn_Xy_original_df[["fraud"]].copy()
knn_X_original_df = knn_Xy_original_df.copy().drop("fraud", axis=1)

# Only for test routie#nes
val_y_originial_df = val_Xy_original_df[["fraud"]].copy()
val_X_originial_df = val_Xy_original_df.copy().drop("fraud", axis=1)

train_y_wo_knn_df = train_Xy_wo_knn_df[["fraud"]].copy()
train_X_wo_knn_df = train_Xy_wo_knn_df.copy().drop("fraud", axis=1)

train_complete_y_originial_df = train_complete_Xy_original_df[["fraud"]].copy()
train_complete_X_originial_df = train_complete_Xy_original_df.copy().drop("fraud", axis=1)

In [21]:
#scaler = MinMaxScaler()
scaler = StandardScaler()
transformer = DataTransformer(scaler)

# Adding new Features to train and test set
train_X_unscaled_df = transformer.add_features(train_X_original_df)
test_X_unscaled_df = transformer.add_features(test_X_original_df)
knn_X_unscaled_df = transformer.add_features(knn_X_original_df)

val_X_unscaled_df = transformer.add_features(val_X_originial_df)
train_complete_X_unscaled_df = transformer.add_features(train_complete_X_originial_df) 

transformer.fit_scaler(transformer.add_features(train_complete_X_unscaled_df.append(test_X_unscaled_df, sort=False)))
train_X_scaled_df = transformer.apply_scaler(train_X_unscaled_df)
knn_X_scaled_df   = transformer.apply_scaler(knn_X_unscaled_df)


test_X_scaled_df  = transformer.apply_scaler(test_X_unscaled_df)
val_X_scaled_df = transformer.apply_scaler(val_X_unscaled_df)
train_complete_X_scaled_df = transformer.apply_scaler(train_complete_X_unscaled_df)

train_X_wo_knn_unscaled_df = transformer.add_features(train_X_wo_knn_df.copy())
train_X_wo_knn_scaled_df = transformer.apply_scaler(train_X_wo_knn_unscaled_df)

# labels
train_y_df = train_y_original_df.copy()
val_y_df = val_y_originial_df.copy()
knn_y_df = knn_y_original_df.copy()

train_complete_X_scaled_df = transformer.apply_scaler(transformer.add_features(train_complete_Xy_original_df.copy().drop(columns=['fraud'])))
train_complete_y_df = train_complete_Xy_original_df.copy().fraud

test_final_X_df = transformer.add_features(test_final_X_df)


display(train_complete_X_scaled_df.head(5))
display(train_complete_y_df.head(5))

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,totalScannedLineItems
0,0.925130,0.240558,0.153259,0.449502,-1.558114,0.300299,-0.117812,-0.128310,-0.371627,1.559426
1,-0.249301,-1.548098,-0.789781,-0.126199,-0.923450,0.886361,0.244945,0.043534,-0.282261,-0.173782
2,-0.249301,1.114087,0.410578,-0.701901,1.615204,1.472424,-0.185091,-0.137604,-0.379817,-0.289330
3,1.512346,1.634045,1.450544,0.737353,-0.288787,0.886361,-0.158033,-0.128614,-0.345007,1.559426
4,0.925130,-0.939274,1.078709,-0.701901,0.663209,-0.285764,0.007505,-0.010833,-0.472189,1.328332


0    0
1    0
2    0
3    0
4    0
Name: fraud, dtype: int64

## Scoring functions
Defining multiple scores which should be tracked in the HyperParamSearch Object

In [11]:
score = scores.Scores()

In [12]:
scoring = {'AUC': 'roc_auc', 'FBeta': metrics.make_scorer(metrics.fbeta_score, beta=0.5172), "Precision":'precision', "Recall": 'recall', "AP": score.average_precision, "DMC" : score.dmc_score}

## Defining the paramteres which should be tuned
To tune the hyperparameters, i looked into the [documentation here](https://xgboost.readthedocs.io/en/latest/parameter.html#parameters-for-tree-booster). `randint` can be used for integer values, for float values, use `uniform`. 

You can also use a Grid search on single parameters to get a feeling for a good interval. If you want to try only two possibilities, you can create a list like for the `scale_pos_weight` parameter.

**Note: For the classifiers which work without gpu support, you can probably set a parameter n_jobs=-1 to use all processors**

In [13]:
params = {
    "tol": uniform(1e-5, 1e-1),  # default 100
    "C" :  uniform(0.0, 80.0),
    "shrinking" : [True, False]
}

## Creating a classifier with some default values
Not all paramters of a classifier should be fine tuned. For SVM for example, the `kernel`-paramter should be set manually. In the case of xgboost, some things like the objective, the booster and the tree method should not be tuned. The choice of paramters depend on the specific classifier

In [15]:
default_svm = SVC(kernel="linear", probability=True, cache_size=8000,  verbose=0, random_state=42)
default_svm.get_params().keys()

dict_keys(['class_weight', 'max_iter', 'shrinking', 'gamma', 'degree', 'decision_function_shape', 'C', 'probability', 'cache_size', 'random_state', 'tol', 'verbose', 'coef0', 'kernel'])

In [22]:
search = RandomizedSearchCV(default_svm, scoring=scoring, param_distributions=params, random_state=42, n_iter=1000,
                            cv=3, verbose=1, n_jobs=-1, return_train_score=True,refit='DMC')
search.fit(train_complete_X_scaled_df, train_complete_y_df)
results = search.cv_results_

Fitting 3 folds for each of 1000 candidates, totalling 3000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 798 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 1698 tasks      | elapsed:   19.7s
[Parallel(n_jobs=-1)]: Done 2798 tasks      | elapsed:   30.9s
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed:   33.0s finished


In [24]:
train_complete_Xy_scaled_df = train_complete_X_scaled_df.copy()
train_complete_Xy_scaled_df['fraud'] =train_complete_y_df.copy()

train_Xy_wo_knn_scaled_df = train_X_wo_knn_scaled_df.copy()
train_Xy_wo_knn_scaled_df['fraud'] = train_y_wo_knn_df

val_Xy_scaled = val_X_scaled_df.copy()
val_Xy_scaled['fraud'] = val_y_df.copy()

In [25]:
scorings = {'AUC': 'roc_auc', 'FBeta': metrics.make_scorer(metrics.fbeta_score, beta=0.5172), "Precision":'precision', "Recall": 'recall', "AP": score.average_precision, "DMC" : score.dmc_score}
xgbo = search.best_estimator_
result_dict = test_classification(xgbo,df_train=train_Xy_wo_knn_scaled_df, df_val=val_Xy_scaled)

Results Fix Split: 
DMC Score: 55  ---  Normalized DMC Score: 0.14627659574468085, 

Results Cross Validation: 
DMC Score: 66.0  ---  Normalized DMC Score: 0.1755886524822695 


In [33]:
dmc = np.mean(search.cv_results_['mean_test_DMC'])
ap = np.mean(search.cv_results_['mean_test_AP'])
precision = np.mean(search.cv_results_['mean_test_Precision'])
recall = np.mean(search.cv_results_['mean_test_Recall'])

In [35]:
print("DMC: {} --- Average Precision: {} --- Precision: {} --- Recall: {}".format(dmc, ap, precision, recall))

DMC: 89.43990420436403 --- Average Precision: 0.9786406695625676 --- Precision: 0.9262799178732629 --- Recall: 0.9497573946449254


In [37]:
search.best_estimator_

SVC(C=11.439334564226868, cache_size=8000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=True, random_state=42,
  shrinking=False, tol=0.08370638742373739, verbose=0)

In [38]:
search.best_params_

{'C': 11.439334564226868, 'shrinking': False, 'tol': 0.08370638742373739}